In [9]:
from preprocess import *
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
conf = SparkConf().setAppName('playground').setMaster('local[*]')
sc = SparkContext(conf=conf)

In [15]:
rules = load_rules()
doclogfreq = load_unigram_cts()
(entid2name, entname2id, entname_set) = load_ent2name()
(doclogfreq, filter_words) = load_unigram_cts()

In [7]:
with open('train.txt','r') as f :
    lines = f.read().split('\n')
    lines = [line.split('\t') for line in lines]

relationship_bags = defaultdict(dict)
    
for line in lines :
    id1 = line[0].replace('www.freebase.com/m','')
    rel_type = line[1].replace('www.freebase.com','')
    id2 = line[2].replace('www.freebase.com/m','')
    text = filter(lambda c: c in [' ','-','\''] or (c>='a' and c<='z') or (c>='0' and c<='9'), line[3].lower())
    text = text.replace('\'',' \'')
    if id1 not in entid2name :
        continue
    ent = entid2name_important[id1]
    new_text = (' '+text+' ').replace(' '+ent+' ',' ')
    words = set(new_text.split()) # TODO using set might be sketchy but dont really want to double count duplicates
    word_weights = dict( zip(words,[1.0/len(words) for i in range(len(words))]) )
    relationship_bags[rel_type] = outer_join_dicts(op.add, relationship_bags[rel_type], word_weights)

In [8]:
# normalize each bag to 1
relationship_bags_norm = {}
relationship_counts = {}
for rel_type in relationship_bags :
    sm = reduce(op.add,relationship_bags[rel_type].values())
    relationship_bags_norm[rel_type] = map_values(lambda x: x/sm, relationship_bags[rel_type])
    relationship_counts[rel_type] = sm

In [10]:
def tfidf_dotproduct(bag1,bag2) :
    score = 0.0
    for key in set.intersection(set(bag1.keys()),set(bag2.keys())) :
        score += bag1[key]*bag2[key]
    return score

In [140]:
with open('test.txt','r') as f :
    lines = f.read().split('\n')
    lines = [line.split('\t') for line in lines]

for line in lines[300:400] :
    id1 = line[0].replace('www.freebase.com/m','')
    rel_true = line[1].replace('www.freebase.com','')
    id2 = line[2].replace('www.freebase.com/m','')
    text = filter(lambda c: c in [' ','-','\''] or (c>='a' and c<='z') or (c>='0' and c<='9'), line[3].lower())
    text = re.sub(r"'(?=[a-z])", r" '", text)
    if id1 not in entid2name :
        print text
        print ':( nonono'
        print ''
        continue
    print text,rel_true,entid2name[id1]
    
    words = text.split()
    unigrams = filter(lambda tup: tup[0] not in filter_words, list(ngrams(words,1)))
    grams_list = unigrams
    for i in range(len(words)) :
        grams_list += list(ngrams(words,i))
    grams = set(grams_list)
    
    possible_ents = list(set.intersection(grams,entname_set))
    P_spelling = [1.0] * len(possible_ents)
    
    # get possible mispellings
    mispelled_ents = []
    for ent in grams-set(possible_ents) :
        if len(' '.join(ent))<=4 or all([doclogfreq[w]>12 for w in ent]) :
            continue
        poss_ents = find_mispellings(ent)
        mispelled_ents += poss_ents
    possible_ents += mispelled_ents
    P_spelling += [0.5] * len(mispelled_ents)
    
    print possible_ents
    print P_spelling
    
    predictions = []
    for ent_i in range(len(possible_ents)) :
        ent_words = possible_ents[ent_i]
        P_ent_len = math.pow(0.5,6-len(ent_words))
        ent_freq = reduce(op.add,[math.log(360000000)-doclogfreq[w] for w in ent_words])
        #print ent_words, P_ent_len, ent_freq
        #new_text = (' '+text+' ').replace(' '+' '.join(ent)+' ',' CHINCHILLA ')
        new_text = (' '+text+' ').replace(' '+' '.join(ent_words)+' ',' ')
        words = set(new_text.split()) # TODO using set might be sketchy but dont really want to double count duplicates
        word_weights = dict( zip(words,[1.0/len(words) for i in range(len(words))]) )
        P_rel = dict([(k,tfidf_dotproduct(word_weights,relationship_bags_norm[k])) for k in relationship_bags_norm])
        #P_rel.sort(lambda p1,p2: int((p2[1]-p1[1])*10000))
        #print P_rel[:10]

        # get all possible rules for entity
        ids = entname2id[' '.join(ent_words)]
        #if len(ids)>100 :
        #    continue
        rules_list = []
        for id in ids :
            rules_list += rules[id]
        #print 'yo',rules_for_id
        for rel in rules_list :
            if rel not in P_rel :
                continue
            #print rel, P_rel[rel]
            predictions.append( (' '.join(ent_words), rel, P_spelling[ent_i]*P_rel[rel]*ent_freq) )
        
    #if id1 not in entid2name_important :
    #    continue
    #ent = entid2name_important[id1]
    #print ent
    #new_text = (' '+text+' ').replace(' '+ent+' ',' ')
    predictions.sort(lambda p1,p2: int(p2[2]*1000-p1[2]*1000))
    #print ''
    for i in range(min(5,len(predictions))) :
        print predictions[i]
    if predictions[0][0]==entid2name[id1] and predictions[0][1]==rel_true :
        print 'yesyesyes'
    else :
        print 'nonono'
    print ''

what language is spoke in lenny /film/film/language lenny
[('in',), ('what',), ('is',), ('spoke',), ('language',), ('lenny',), ('dino', 'lenny'), ('spoken',)]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 0.5]
('lenny', '/film/film/language', 1.300816472662362)
('lenny', '/book/written_work/original_language', 1.0582204055666007)
('lenny', '/book/short_story/genre', 0.8062899978960012)
('lenny', '/film/film/genre', 0.7774189025010032)
('dino lenny', '/people/person/gender', 0.7602668202446907)
yesyesyes

where was kelly deadmon born /people/person/place_of_birth kelly deadmon
[('kelly',), ('kelly', 'deadmon'), ('where',), ('born',), ('was',), ('dead', 'on'), ('deadman',), ('dead', 'moon'), ('demon',), ('sam', 'kelly')]
[1.0, 1.0, 1.0, 1.0, 1.0, 0.5, 0.5, 0.5, 0.5, 0.5]
('kelly deadmon', '/people/person/place_of_birth', 5.806455123109734)
('sam kelly', '/people/person/place_of_birth', 1.0467677649527996)
('demon', '/people/person/place_of_birth', 0.5905180958864773)
('kelly', '/music/release/regi

In [ ]:
14 unsolvable/weird/indistinguishable
1 ent
16 exact lm (credit for full match or single word of two in any part of rel, sometimes plural vs singular)
9 lm (almost always synonym)

In [ ]:
59 correct
17 ?ent
19 nope5
5 nopeno (almost all really similar rules chosen)

In [ ]:
x13 common ent
carlos gomez, pretty in pink, somevelvetsidewalk
-2 not exact match (ED: 1,2)

In [35]:
for line in lines[:100] :
    id1 = line[0].replace('www.freebase.com/m','')
    print entid2name_important[id1] if id1 in entid2name_important else ':('

harderfaster
fearless
alex golfis
phil hay
roger marquis
yves klein
dublin
action
las vegas
akbar tandjung
jean grae
carlos gmez
engelbert zaschka
carolyn s shoemaker
pretty in pink
woodstock villa
shawn holly cookson
:(
pee wee reese
one life to live
stephen mao
carpenter
myocardial infarction
david ruffin
equator
los angeles
history
padraic mcguinness
viper
environmental disaster
magical mystery tour
michael mcdowell
the complete on the corner sessions
michael dotson
sam edwards
second battle of fort fisher
pembroke welsh corgi
music for cello orchestra
voodoo lounge
some velvet sidewalk
tokyo
italianate architecture
loud
neo contra
western
hindi language
aguascalientes
herb alpert
angie estes
indiana
jeff hall
jos francisco torres
philip leacock
chic murray
the soul sessions
frei otto
cow belles
around the world
a knot in the grain and other stories
dancin' homer
british columbia
primero river
johannes messenius
the house that never was
roman italy
taffin
warsaw
philharmonia orchest

In [87]:
sc.textFile("freebase-rules.txt").map(process_rule_line).distinct().coalesce(1).saveAsTextFile("rules")

In [82]:
x.count()

1980225

In [99]:
rules['/02ppyz8']

['/people/person/profession',
 '/people/person/nationality',
 '/people/person/gender',
 '/people/person/place_of_birth']

In [118]:
primes_letters = [2,3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,67,71,73,79,83,89,97,101]
primes_numbers = [103,109,113,127,131,137,139,149,151,157]
primes_all = primes_letters + primes_numbers + [163,167,173]
possible_spelling_ratios = set( flatten_one_layer([[1.0*x*y,1.0*x/y,1.0*y/x,1.0/x/y] for x in primes_all for y in primes_all])
        + flatten_one_layer([[1.0*x,1.0/x] for x in primes_all]) )
primes_map = {' ':163,'-':167,'\'':173}
for i in range(26) :
    primes_map[chr(ord('a')+i)] = primes_letters[i]
for i in range(10) :
    primes_map[chr(ord('0')+i)] = primes_numbers[i]
    
ent_spell_scores = {}
for ent in entname_set :
    num_list = [primes_map[c] for c in ' '.join(ent)]
    if len(num_list)==0 or len(num_list)>40 :
        continue
    ent_spell_scores[float(reduce(op.mul,num_list))] = ent

In [82]:
def flatten_one_layer(l) :
    return [item for sublist in l for item in sublist]

In [94]:
def edit_distance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [122]:
def find_mispellings(ent) :
    find_val = reduce(op.mul,[primes_map[c] for c in ' '.join(ent)])
    possibilities = []
    for ratio in possible_spelling_ratios :
        if find_val*ratio in ent_spell_scores :
            possibilities.append(ent_spell_scores[long(find_val*ratio)])

    found = []
    for poss in possibilities :
        if edit_distance(' '.join(poss),' '.join(ent))<=2 :
            found.append(poss)
    return found

In [123]:
find_mispellings(('somevelvetsidewalk',))

[('some', 'velvet', 'sidewalk')]

In [104]:
len(possible_composites)

780

In [97]:
ent_spell_scores[20911329313528897743071110L]

KeyError: 20911329313528897743071110L

In [98]:
num_list = [primes_map[c] for c in ' '.join(('skyclan\'s','destiny',))]
reduce(op.mul,num_list)

212803527720029371150076590L

In [99]:
num_list = [primes_map[c] for c in ' '.join(('skyclans','destiny',))]
reduce(op.mul,num_list)

1230078194913464573121830L

In [115]:
1230078194913464573121830L/173

7110278583314824122091L

In [129]:
doclogfreq['until'],doclogfreq['areas'],doclogfreq['gomez'],doclogfreq['random']

(13.162954305230732, 12.283945308050269, 8.902591637374087, 10.364860292582588)